In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
import math
import torch
from torch.optim.optimizer import Optimizer, required

class RAdam(Optimizer):

    def __init__(self, params, lr=1e-3, betas=(0.9, 0.999), eps=1e-8, weight_decay=0):
        defaults = dict(lr=lr, betas=betas, eps=eps, weight_decay=weight_decay)
        self.buffer = [[None, None, None] for ind in range(10)]
        super(RAdam, self).__init__(params, defaults)

    def __setstate__(self, state):
        super(RAdam, self).__setstate__(state)

    def step(self, closure=None):

        loss = None
        if closure is not None:
            loss = closure()

        for group in self.param_groups:

            for p in group['params']:
                if p.grad is None:
                    continue
                grad = p.grad.data.float()
                if grad.is_sparse:
                    raise RuntimeError('RAdam does not support sparse gradients')

                p_data_fp32 = p.data.float()

                state = self.state[p]

                if len(state) == 0:
                    state['step'] = 0
                    state['exp_avg'] = torch.zeros_like(p_data_fp32)
                    state['exp_avg_sq'] = torch.zeros_like(p_data_fp32)
                else:
                    state['exp_avg'] = state['exp_avg'].type_as(p_data_fp32)
                    state['exp_avg_sq'] = state['exp_avg_sq'].type_as(p_data_fp32)

                exp_avg, exp_avg_sq = state['exp_avg'], state['exp_avg_sq']
                beta1, beta2 = group['betas']

                exp_avg_sq.mul_(beta2).addcmul_(1 - beta2, grad, grad)
                exp_avg.mul_(beta1).add_(1 - beta1, grad)

                state['step'] += 1
                buffered = self.buffer[int(state['step'] % 10)]
                if state['step'] == buffered[0]:
                    N_sma, step_size = buffered[1], buffered[2]
                else:
                    buffered[0] = state['step']
                    beta2_t = beta2 ** state['step']
                    N_sma_max = 2 / (1 - beta2) - 1
                    N_sma = N_sma_max - 2 * state['step'] * beta2_t / (1 - beta2_t)
                    buffered[1] = N_sma

                    # more conservative since it's an approximated value
                    if N_sma >= 5:
                        step_size = math.sqrt((1 - beta2_t) * (N_sma - 4) / (N_sma_max - 4) * (N_sma - 2) / N_sma * N_sma_max / (N_sma_max - 2)) / (1 - beta1 ** state['step'])
                    else:
                        step_size = 1.0 / (1 - beta1 ** state['step'])
                    buffered[2] = step_size

                if group['weight_decay'] != 0:
                    p_data_fp32.add_(-group['weight_decay'] * group['lr'], p_data_fp32)

                # more conservative since it's an approximated value
                if N_sma >= 5:            
                    denom = exp_avg_sq.sqrt().add_(group['eps'])
                    p_data_fp32.addcdiv_(-step_size * group['lr'], exp_avg, denom)
                else:
                    p_data_fp32.add_(-step_size * group['lr'], exp_avg)

                p.data.copy_(p_data_fp32)

        return loss

In [3]:
import tqdm
import numpy as np 
import pandas as pd

from PIL import Image

import torch
import torch.nn as nn
import torch.utils.data as D
import torch.nn.functional as F

import albumentations
import torchvision
from torchvision import transforms

from matplotlib import pyplot as plt
from tqdm import tqdm_notebook
from sklearn.model_selection import StratifiedKFold, KFold, train_test_split

import cv2
import numpy as np
import pandas as pd

import torch.utils.data as D
from torchvision import transforms as T


class ImagesDS(D.Dataset):
    def __init__(self, csv_file, img_dir, site=None,
                 transforms=None, mode='train',
                 channels=[1, 2, 3, 4, 5, 6]):
        # df = pd.read_csv(csv_file)
        self.records = csv_file.to_records(index=False)
        self.channels = channels
        self.site = site
        self.mode = mode
        self.transforms = transforms
        self.img_dir = img_dir
        if mode == 'train':
            self.len = csv_file.shape[0] * 2
        elif self.mode == 'test':
            self.len = csv_file.shape[0]

    def _get_img_path(self, index, site, channel):
        exp, well, plate = self.records[index].experiment, self.records[index].well, self.records[index].plate
        return '/'.join([self.img_dir, self.mode, exp, 'Plate{}'.format(plate),
                         '{}_s{}_w{}.png'.format(well, site, channel)])

    def __getitem__(self, index):
        if self.mode == 'train':
            site = (index % 2) + 1
            index = index // 2
        elif self.mode == 'test':
            site = self.site
            index = index

        paths = [self._get_img_path(index, site, ch) for ch in self.channels]
        img = [cv2.imread(img_path, cv2.IMREAD_GRAYSCALE) for img_path in paths]

        if self.transforms is None:
            # img = torch.cat([T.ToTensor()(i) for i in img])
            img = T.ToTensor()(np.dstack([img]))
        else:
            # img = self.transforms(torch.cat([T.ToTensor()(i) for i in img]))
            img = T.ToTensor()(self.transforms(image=np.dstack(img))['image'])

        if self.mode == 'train':
            return img, self.records[index].sirna
        elif self.mode == 'test':
            return img, self.records[index].id_code

    def __len__(self):
        return self.len

In [4]:
def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred    = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))

    return res

In [5]:
path_data = '/home/vladislav/Data'
device = 'cuda'
batch_size = 24

In [6]:
train_df = pd.read_csv(path_data+'/train.csv')
ctrl_df = pd.read_csv(path_data+'/train_controls.csv')[['id_code', 'experiment', 'plate', 'well', 'sirna']]
train_df = pd.concat([train_df, ctrl_df])
X_train, X_test, _, _ = train_test_split(train_df, train_df['sirna'],
                                         random_state=38, 
                                         test_size=0.1, 
                                         stratify=train_df['sirna'])
X_train.shape, X_test.shape

((36550, 5), (4062, 5))

In [7]:
pix_df = pd.read_csv(path_data+'/pixel_stats.csv')
means, stds = [], []
for ch in [1, 2, 3, 4, 5, 6]:
    means.append((pix_df[pix_df['channel']==ch][['mean']].mean()/255.)[0])
    stds.append((pix_df[pix_df['channel']==ch][['std']].mean()/255.)[0])

In [54]:
pd.read_csv(path_data+'/test_controls.csv')

,id_code,experiment,plate,well,sirna,well_type
0,HEPG2-08_1_B02,HEPG2-08,1,B02,1138,negative_control
1,HEPG2-08_1_C03,HEPG2-08,1,C03,1137,positive_control
2,HEPG2-08_1_C07,HEPG2-08,1,C07,1120,positive_control
3,HEPG2-08_1_C11,HEPG2-08,1,C11,1108,positive_control
4,HEPG2-08_1_C15,HEPG2-08,1,C15,1113,positive_control
5,HEPG2-08_1_C19,HEPG2-08,1,C19,1124,positive_control
6,HEPG2-08_1_C22,HEPG2-08,1,C22,1133,positive_control
7,HEPG2-08_1_F03,HEPG2-08,1,F03,1109,positive_control
8,HEPG2-08_1_F07,HEPG2-08,1,F07,1116,positive_control
9,HEPG2-08_1_F11,HEPG2-08,1,F11,1135,positive_control


In [53]:
pd.read_csv(path_data+'/train_controls.csv')

,id_code,experiment,plate,well,sirna,well_type
0,HEPG2-01_1_B02,HEPG2-01,1,B02,1138,negative_control
1,HEPG2-01_1_C03,HEPG2-01,1,C03,1109,positive_control
2,HEPG2-01_1_C07,HEPG2-01,1,C07,1121,positive_control
3,HEPG2-01_1_C11,HEPG2-01,1,C11,1126,positive_control
4,HEPG2-01_1_C15,HEPG2-01,1,C15,1118,positive_control
5,HEPG2-01_1_C19,HEPG2-01,1,C19,1116,positive_control
6,HEPG2-01_1_C22,HEPG2-01,1,C22,1110,positive_control
7,HEPG2-01_1_F03,HEPG2-01,1,F03,1113,positive_control
8,HEPG2-01_1_F07,HEPG2-01,1,F07,1114,positive_control
9,HEPG2-01_1_F11,HEPG2-01,1,F11,1137,positive_control


In [60]:
for i, j in pd.read_csv(path_data+'/test_controls.csv').iterrows():
    print(j)
    break

id_code         HEPG2-08_1_B02
experiment            HEPG2-08
plate                        1
well                       B02
sirna                     1138
well_type     negative_control
Name: 0, dtype: object


In [62]:
from shutil import copyfile

for i, j in pd.read_csv(path_data+'/test_controls.csv').iterrows():
    os.mkdir('')
    for site in [1, 2]:
        for channel in [1, 2, 3, 4, 5, 6]:
            src = '/'.join([path_data, 'test', j['experiment'], 'Plate{}'.format(j['plate']),
                                 '{}_s{}_w{}.png'.format(j['well'], site, channel)])
            dst = '/'.join([path_data, 'train', j['experiment'], 'Plate{}'.format(j['plate']),
                         '{}_s{}_w{}.png'.format(j['well'], site, channel)])
            print(src, dst)
            # copyfile
    break

/home/vladislav/Data/test/HEPG2-08/Plate1/B02_s1_w1.png /home/vladislav/Data/train/HEPG2-08/Plate1/B02_s1_w1.png
/home/vladislav/Data/test/HEPG2-08/Plate1/B02_s1_w2.png /home/vladislav/Data/train/HEPG2-08/Plate1/B02_s1_w2.png
/home/vladislav/Data/test/HEPG2-08/Plate1/B02_s1_w3.png /home/vladislav/Data/train/HEPG2-08/Plate1/B02_s1_w3.png
/home/vladislav/Data/test/HEPG2-08/Plate1/B02_s1_w4.png /home/vladislav/Data/train/HEPG2-08/Plate1/B02_s1_w4.png
/home/vladislav/Data/test/HEPG2-08/Plate1/B02_s1_w5.png /home/vladislav/Data/train/HEPG2-08/Plate1/B02_s1_w5.png
/home/vladislav/Data/test/HEPG2-08/Plate1/B02_s1_w6.png /home/vladislav/Data/train/HEPG2-08/Plate1/B02_s1_w6.png
/home/vladislav/Data/test/HEPG2-08/Plate1/B02_s2_w1.png /home/vladislav/Data/train/HEPG2-08/Plate1/B02_s2_w1.png
/home/vladislav/Data/test/HEPG2-08/Plate1/B02_s2_w2.png /home/vladislav/Data/train/HEPG2-08/Plate1/B02_s2_w2.png
/home/vladislav/Data/test/HEPG2-08/Plate1/B02_s2_w3.png /home/vladislav/Data/train/HEPG2-08/Plat

In [8]:
transforms_train = albumentations.Compose([
    albumentations.RandomCrop(height=384, width=384, p=1),
    albumentations.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=0, p=0.3),
    albumentations.RandomRotate90(p=0.3),
    albumentations.Flip(p=0.3),
    albumentations.Normalize(
     mean=means,
     std=stds)
])

transforms_test = albumentations.Compose([
    albumentations.RandomCrop(height=384, width=384, p=1),
    albumentations.Normalize(
     mean=means,
     std=stds)
])

In [9]:
ds = ImagesDS(X_train, path_data, None, transforms_train, mode='train')
ds_valid = ImagesDS(X_test, path_data, None, transforms_test, mode='train')


loader = D.DataLoader(ds, batch_size=batch_size, shuffle=True, num_workers=1)
loader_valid = D.DataLoader(ds_valid, batch_size=batch_size, shuffle=True, num_workers=1)

In [10]:
class DenseNet(nn.Module):
    def __init__(self, num_classes=1139, num_channels=6):
        super().__init__()
        preloaded = torchvision.models.densenet121(pretrained=True)
        self.features = preloaded.features
        self.features.conv0 = nn.Conv2d(num_channels, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        self.dp = nn.Dropout(0.4)
        self.classifier = nn.Linear(1024, num_classes, bias=True)
        del preloaded
        
    def forward(self, x):
        features = self.features(x)
        out = F.adaptive_avg_pool2d(features, (1, 1)).view(features.size(0), -1)
        out = self.dp(out)
        out = self.classifier(out)
        return out

In [11]:
model = DenseNet()
model.to(device);

In [12]:
criterion = nn.BCEWithLogitsLoss()
optimizer = RAdam(model.parameters(), lr=30e-4)

epochs = 36
best_acc = 0
tlen = len(loader)
for epoch in range(epochs):
    tloss = 0
    acc = np.zeros(1)
    model.train()
    tlen = len(loader)
    for x, y in tqdm.tqdm_notebook(loader):
        x = x.to(device)
        optimizer.zero_grad()
        output = model(x)
        target = torch.zeros_like(output, device=device)
        target[np.arange(x.size(0)), y] = 1
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        tloss += loss.item() 
        acc += accuracy(output.cpu(), y)
        del loss, output, y, x, target
    if epoch > 10:
        optimizer.param_groups[0]['lr'] = 15e-4
    if epoch > 20:
        optimizer.param_groups[0]['lr'] = 7.5e-4
    if epoch > 25:
        optimizer.param_groups[0]['lr'] = 3e-4
    if epoch > 30:
        optimizer.param_groups[0]['lr'] = 1e-4
    print('Epoch {} -> Train Loss: {:.4f}, ACC: {:.2f}%'.format(epoch+1, tloss/tlen, acc[0]/tlen))

    acc = np.zeros(1)
    tlen = len(loader_valid)
    model.eval()
    for x, y in loader_valid:
        x = x.to(device)
        output = model(x)
        acc += accuracy(output.cpu(), y)
        del output, y, x
    print('Epoch {} -> Valid ACC: {:.2f}%'.format(epoch+1, acc[0]/tlen))
    
    if acc[0] >= best_acc:
        best_acc = acc[0]
        torch.save(model.state_dict(), 'dn_pretrain.pth')


Epoch 1 -> Train Loss: 0.0139, ACC: 0.66%
Epoch 1 -> Valid ACC: 0.92%



Epoch 2 -> Train Loss: 0.0067, ACC: 2.00%
Epoch 2 -> Valid ACC: 2.83%



Epoch 3 -> Train Loss: 0.0062, ACC: 4.69%
Epoch 3 -> Valid ACC: 4.63%



Epoch 4 -> Train Loss: 0.0058, ACC: 8.34%
Epoch 4 -> Valid ACC: 6.48%



Epoch 5 -> Train Loss: 0.0054, ACC: 12.88%
Epoch 5 -> Valid ACC: 8.14%


...


Epoch 18 -> Train Loss: 0.0030, ACC: 52.79%
Epoch 18 -> Valid ACC: 28.98%



Epoch 19 -> Train Loss: 0.0030, ACC: 54.05%
Epoch 19 -> Valid ACC: 28.86%



Epoch 20 -> Train Loss: 0.0029, ACC: 55.14%
Epoch 20 -> Valid ACC: 33.38%



Epoch 21 -> Train Loss: 0.0029, ACC: 56.17%
Epoch 21 -> Valid ACC: 21.07%


KeyboardInterrupt: 

---

In [19]:
ds_test = ImagesDS(pd.read_csv(path_data+'/test.csv'), path_data, 2, transforms_test, mode='test')
tloader = D.DataLoader(ds_test, batch_size=batch_size, shuffle=False, num_workers=8)

In [20]:
@torch.no_grad()
def prediction(model, loader):
    preds = [] # np.empty(0)
    for x, _ in tqdm_notebook(loader): 
        x = x.to(device)
        output = model(x)[:1108]
        # idx = output.max(dim=-1)[1].cpu().numpy()
        # preds = np.append(preds, idx, axis=0)
        preds.append(output.cpu().numpy())
    return preds

preds2 = prediction(model, tloader)

In [48]:
preds = ((np.concatenate(preds)+np.concatenate(preds2)) / 2.)[:, :1108].argmax(axis=1)

In [49]:
submission = pd.read_csv(path_data + '/test.csv')
submission['sirna'] = preds.astype(int)
submission.to_csv('submission.csv', index=False, columns=['id_code','sirna'])

Epoch 20 -> Valid ACC: 33.38% | 0.312